In [1]:
import re, random, concurrent.futures as cf
from datasets import load_dataset, Dataset
from litellm import completion
from huggingface_hub import login, create_repo
from tqdm import tqdm 

In [2]:
REPO_ID     = "ping98k/squad-qwq-recall-1k"
MODEL       = "hosted_vllm//home/ping/qwq-32b-q4_k_m.gguf"
URL  = "http://localhost:8889/v1"
API_KEY = "token-abc123"
NUM_SAMPLES = 4
MAX_WORKERS = 4 
TEMPERATURE = 0.8


think_re     = re.compile(r"<think>(.*?)</think>", re.DOTALL)
knowledge_re = re.compile(r"\[knowledge\](.*?)\[/knowledge\]", re.DOTALL)

def _single_completion(q):

    prompt = f"""
Recall as much knowledge as you can to prepare for any upcoming question.
You may only recall knowledge during this time.
After this, you can only use the knowledge you have recalled.
Here is an example question: {q}
Do not answer the question. Instead, recall any knowledge related to it.
Wrap all recalled knowledge within [knowledge]...[/knowledge].
""".strip()
    resp = completion(
        model=MODEL,
        messages=[{"role": "user", "content": prompt}],
        api_base=URL,
        temperature=TEMPERATURE,
        api_key=API_KEY,
        n=NUM_SAMPLES,                    
    )

    samples = []
    for choice in resp.choices:
        txt = choice.message.content.strip()

        m = knowledge_re.search(txt)
        knowledge = m.group(1).strip() if m else ""

        t = think_re.search(txt)
        think = t.group(1).strip() if t else ""

        samples.append({"knowledge": knowledge, "think": think})

    return samples


try:
    existing = load_dataset(REPO_ID, split="train")
    processed_ids = set(existing["id"])
    rows = [existing[i] for i in range(len(existing))]
except Exception:
    processed_ids = set()
    rows = []

src = (
    load_dataset("rajpurkar/squad", split="train")
    .shuffle(seed=42)
    .select(range(1_000))
)

def build_row(example):
    if example["id"] in processed_ids:
        return None
    return {
        "id": example["id"],
        "question": example["question"],
        "recall": _single_completion(example["question"]),  # ← direct call
    }

for ex in tqdm(src, total=len(src), desc="Building rows"):
    if ex["id"] in processed_ids:
        continue

    row = build_row(ex)          # ← direct call, no executor
    if row is None:
        continue

    rows.append(row)
    processed_ids.add(row["id"])

    ds = Dataset.from_dict({
        "id":        [r["id"]       for r in rows],
        "question":  [r["question"] for r in rows],
        "recall":    [r["recall"]   for r in rows],
    })
    ds.push_to_hub(
        repo_id=REPO_ID,
        split="train",
        commit_message=f"Add {row['id']}"
    )

README.md:   0%|          | 0.00/417 [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/9.26M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/817 [00:00<?, ? examples/s]

Building rows:  77%|████████████████████████████████████████████████▋              | 773/1000 [00:12<00:00, 4218.87it/s]

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

...

Building rows: 100%|█████████████████████████████████████████████████████████████▉| 999/1000 [6:30:48<02:35, 155.46s/it]

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

README.md:   0%|          | 0.00/418 [00:00<?, ?B/s]

Building rows: 100%|██████████████████████████████████████████████████████████████| 1000/1000 [6:32:41<00:00, 23.56s/it]


In [3]:
# # MODEL       = "hosted_vllm//home/ping/qwq-32b-q4_k_m.gguf"
# MODEL       = "openai/qwq-32b-q4_k_m.gguf"
# URL  = "http://192.168.1.111:8889/v1"
# # URL  = "http://localhost:8889/v1"

# API_KEY = "token-abc123"
# NUM_SAMPLES = 4
# TEMPERATURE = 0.8

# res = completion(
#         model=MODEL,
#         messages=[{"role": "user", "content": "hi"}],
#         api_base=URL,
#         temperature=TEMPERATURE,
#         api_key=API_KEY,
#         n=4
#     )
# res